# Homework 1 Part 4

## Course Name: Large Language Models
#### Lecturers: Dr. Soleimani, Dr. Rohban, Dr. Asgari

---

#### Notebooks Supervised By: MohammadAli SadraeiJavaheri
#### Notebooks Prepared By: Zeinab Sadat Taghavi, Hamed Jamshidian, Seyed Mohammad Reza Modarres

**Contact**: Ask your questions in Quera

---

### Instructions:
- Complete all exercises presented in this notebook.
- Ensure you run each cell after you've entered your solution.
- After completing the exercises, save the notebook and <font color='red'>follow the submission guidelines provided in the PDF.</font>


---

**Note**: Replace the placeholders (between `## Your code begins ##` and `## Your code ends ##`) with the appropriate details.


## Introduction

In this notebook you have to use [Adapter Hub](https://docs.adapterhub.ml/overview.html) to define and train your adapters.

### Requirements

In [ ]:
%%capture
! pip install datasets adapter-transformers

### Imports

In [ ]:
from tqdm.notebook import tqdm
from IPython import display

import numpy as np
import pandas as pd
import math

from sklearn.metrics import accuracy_score

import torch
import torch.nn as nn

from datasets import load_dataset
from transformers import T5TokenizerFast, T5ForConditionalGeneration, DataCollatorForSeq2Seq
from transformers.models.t5.modeling_t5 import T5LayerFF

### Constants

We will use `t5-small` as our base model from Hugging Face ([HF_Link](https://huggingface.co/t5-small)). And we use `32` as our adapter bottleneck size.

In [ ]:
#####################################
###### DO NOT CHANGE THIS CELL ######
#####################################

BASE_MODEL_NAME = 't5-small'

BATCH_SIZE = 32
LEARNING_RATE = 1e-4
EPOCHS = 10

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## Dataset

### Load dataset

`imdb` dataset is a famouns NLP sentiment dataset. Each row of data is either `negative` or `positive`.

In [ ]:
dataset = load_dataset('imdb')
dataset.pop('unsupervised')
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
})


### Define related functions

Because `T5` model is a sequence to sequence model we should map our labels to label_names before training and doing vice versa duing calculating metrics.

The functions `id2label` and `label2id` are defined to do this.

In [ ]:
def id2label(ids):
    label_names = ['negative', 'positive']
    return [label_names[id] for id in ids]

def label2id(labels):
    label_names_dict = {
        'negative': 0,
        'positive': 1
    }
    return [
        label_names_dict.get(label, 2)
        for label in labels
    ]

# Tokenizer

### Load tokenizer

In [ ]:
tokenizer = T5TokenizerFast.from_pretrained(BASE_MODEL_NAME)

### Process dataset using tokenizer

In this step we will getting our dataset ready for training.

We preprocess tokenize our `text` and `label`.

In [ ]:
def preprocess_input(text):
    text = text.lower()
    text = text.replace('<br />', ' ')
    return text

def map_function(row):
    processed_input = [
        preprocess_input(text)
        for text in row['text']
    ]
    input_info = tokenizer(processed_input, truncation=True, max_length=256)
    output_info = tokenizer(id2label(row['label']))
    return {
        **input_info,
        'labels': output_info.input_ids
    }


dataset = dataset.map(map_function, batched=True)
dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

# Model

### Load model and create adapter

In next part create `PfeifferConfig` by considering `BOTTLENECK_SIZE`.

Complete the next cell using methods of `train_adapter` and `add_adapter`.

Report final test data performance.

Read [this page](https://docs.adapterhub.ml/training.html) for more information.

In [ ]:
from transformers import PfeifferConfig

BOTTLENECK_SIZE = 8

model = T5ForConditionalGeneration.from_pretrained(BASE_MODEL_NAME)

######### Your code begins #########
reduction_factor = model.config.d_model / BOTTLENECK_SIZE
adapter_config = PfeifferConfig(reduction_factor=reduction_factor)
model.add_adapter("Pfeiffer", config=adapter_config)
model.train_adapter("Pfeiffer")
model.set_active_adapters('Pfeiffer')
####################################
####################################

# Train and evaluate

Complete next part to train your peft using `AdapterTrainer`.

In [ ]:
from transformers import TrainingArguments, AdapterTrainer

col_fn = DataCollatorForSeq2Seq(
    tokenizer, return_tensors='pt', padding='longest',
)

training_args = TrainingArguments(
    './temp',
    learning_rate=LEARNING_RATE,
    num_train_epochs=EPOCHS,
    logging_strategy='epoch',
    evaluation_strategy='epoch',
    save_strategy='epoch',
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE
)

######### Your code begins #########
adapter_trainer = AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    data_collator=col_fn,
    adapter_names=["Pfeiffer"],
)

adapter_trainer.train()
####################################
####################################

In [ ]:
col_fn = DataCollatorForSeq2Seq(
    tokenizer, return_tensors='pt', padding='longest',
)

train_loader = torch.utils.data.DataLoader(
    dataset['train'],
    batch_size=BATCH_SIZE,
    collate_fn=col_fn,
    shuffle=True
)

test_loader = torch.utils.data.DataLoader(
    dataset['test'],
    batch_size=BATCH_SIZE,
    collate_fn=col_fn
)

In [ ]:
def train_loop(model, loader, optimizer):
    model.train()

    batch_losses = []

    for row in tqdm(loader, desc='Training:'):
        optimizer.zero_grad()

        out = model(**row.to(model.device))
        loss = out.loss

        batch_loss_value = loss.item()
        loss.backward()
        optimizer.step()

        batch_losses.append(batch_loss_value)

    loss_value = np.mean(batch_losses)
    return {'train_loss': loss_value}

def _predict(model, row):
    return model.generate(
        input_ids=row.input_ids,
        attention_mask=row.attention_mask,
        max_length=5
    )

def tokenizer_ids_to_label(all_input_ids):
    return tokenizer.batch_decode(all_input_ids, skip_special_tokens=True)

def valid_loop(model, loader, compute_metrics):
    model.eval()

    all_true = []
    all_pred = []

    with torch.no_grad():
        for row in tqdm(loader, desc='Validating:'):
            row.to(model.device)
            pred = _predict(model, row)

            all_true += row.labels.detach().cpu().tolist()
            all_pred += pred.detach().cpu().tolist()

    all_true = label2id(tokenizer_ids_to_label(all_true))
    all_pred = label2id(tokenizer_ids_to_label(all_pred))

    return {'valid_acc': compute_metrics(y_true=all_true, y_pred=all_pred)}

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)
compute_metrics = accuracy_score

In [ ]:
model.to(DEVICE)

all_results = []
for epoch in range(EPOCHS):
    epoch_results = {'epoch': epoch}

    epoch_results.update(
        train_loop(
            model=model,
            loader=train_loader,
            optimizer=optimizer,
        )
    )

    epoch_results.update(
        valid_loop(
            model=model,
            loader=test_loader,
            compute_metrics=compute_metrics,
        )
    )
    all_results.append(epoch_results)

    display.clear_output()
    display.display(pd.DataFrame(all_results).set_index('epoch'))

,train_loss,valid_acc
epoch,,
0,0.667763,0.86220
1,0.163000,0.88260
2,0.150823,0.88848
3,0.144641,0.89356
4,0.140519,0.89732
5,0.135077,0.89972
6,0.133471,0.90080
7,0.129795,0.89960
8,0.127145,0.89944


In [5]:
best_score = pd.DataFrame(all_results)['valid_acc'].max() * 100
total_params = sum(p.numel() for p in model.parameters())
print(f"Number of parameters: {total_params}")
print(f'Best model preformance given bottleneck_size = {BOTTLENECK_SIZE} is: %%%.1f' % best_score)

Number of parameters: 60611168
Best model preformance given bottleneck_size = 8 is: %90.1


# Bottleneck effect

Check the model performance with `BOTTLENECK_SIZE = 1` and report the number.

In [ ]:
from transformers import PfeifferConfig

BOTTLENECK_SIZE = 1

model = T5ForConditionalGeneration.from_pretrained(BASE_MODEL_NAME)

######### Your code begins #########
reduction_factor = model.config.d_model / BOTTLENECK_SIZE
adapter_config = PfeifferConfig(reduction_factor=reduction_factor)
model.add_adapter("Pfeiffer", config=adapter_config)
model.train_adapter("Pfeiffer")
model.set_active_adapters('Pfeiffer')
####################################
####################################

In [ ]:
from transformers import TrainingArguments, AdapterTrainer

col_fn = DataCollatorForSeq2Seq(
    tokenizer, return_tensors='pt', padding='longest',
)

training_args = TrainingArguments(
    './temp',
    learning_rate=LEARNING_RATE,
    num_train_epochs=EPOCHS,
    logging_strategy='epoch',
    evaluation_strategy='epoch',
    save_strategy='epoch',
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE
)

######### Your code begins #########
adapter_trainer = AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    data_collator=col_fn,
    adapter_names=["Pfeiffer"],
)

adapter_trainer.train()
####################################
####################################

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)
compute_metrics = accuracy_score

In [ ]:
model.to(DEVICE)

all_results = []
for epoch in range(EPOCHS):
    epoch_results = {'epoch': epoch}

    epoch_results.update(
        train_loop(
            model=model,
            loader=train_loader,
            optimizer=optimizer,
        )
    )

    epoch_results.update(
        valid_loop(
            model=model,
            loader=test_loader,
            compute_metrics=compute_metrics,
        )
    )
    all_results.append(epoch_results)

    display.clear_output()
    display.display(pd.DataFrame(all_results).set_index('epoch'))

,train_loss,valid_acc
epoch,,
0,1.504263,0.80676
1,0.229627,0.84432
2,0.186046,0.86280
3,0.173076,0.86992
4,0.164625,0.87676
5,0.159747,0.88284
6,0.154321,0.88656
7,0.149370,0.88844
8,0.146632,0.88880


In [3]:
best_score = pd.DataFrame(all_results)['valid_acc'].max() * 100
total_params = sum(p.numel() for p in model.parameters())
print(f"Number of parameters: {total_params}")
print(f'Best model preformance given bottleneck_size = {BOTTLENECK_SIZE} is: %%%.1f' % best_score)

Number of parameters: 60525068
Best model preformance given bottleneck_size = 1 is: %89.1
